# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
import base64
def get_key(keyfile='config.py'):
    with open(keyfile,'rb') as inf:
        k = base64.b64decode(inf.read())
        inf.close()
    return k.decode('utf8')

gkey=get_key()

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
#Create a path and load it into dataframe
path = ("../WeatherPy/city_data.csv")
city_data = pd.read_csv(path)

#Drop unnecessary colummn
city_data = city_data.drop(columns=["Unnamed: 0"])
city_data

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,hamilton,1,US,1585015854,70,39.18,-84.53,45.00,6.93
1,hambantota,0,LK,1585015854,100,6.12,81.12,73.40,3.36
2,busselton,89,AU,1585015854,29,-33.65,115.33,82.00,6.04
3,hobart,20,AU,1585015801,47,-42.88,147.33,64.99,8.05
4,mataura,100,NZ,1585015854,90,-46.19,168.86,52.00,15.01
...,...,...,...,...,...,...,...,...,...
551,qadian,75,IN,1585015937,82,31.82,75.39,66.20,2.24
552,haradok,0,BY,1585015937,55,55.46,29.98,24.98,3.38
553,pafos,0,CY,1585015937,75,34.77,32.42,53.60,6.93
554,margate,0,GB,1585015937,64,51.38,1.39,46.00,10.29


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#Configure the maaaAaaaAAAaaAAAaaaps (said in voice of the Yeah Yeah Yeah's "Maps")
gmaps.configure(api_key=gkey)

In [4]:
#Create variables and initial fig to be used in heatmap
locations = city_data[["Lat", "Lng"]]
humidity = city_data["Humidity"].astype(float)
fig = gmaps.figure()

#Add some heat
heat_layer = gmaps.heatmap_layer(locations, 
                                 weights=humidity, 
                                 dissipating=False, 
                                 max_intensity=(max(humidity)+1), 
                                 point_radius=2)
figure_layout = {
'width': '1000px',
'height': '600px',
'border': '1px solid black',
'padding': '1px',
'margin': '0 auto 0 auto'}
fig = gmaps.figure(zoom_level=2, center=(0.0, 0.0), layout=figure_layout)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
#Narrow down cities to fit ideal weather
vacation = city_data.loc[(city_data["Max Temp"] < 80) &
                        (city_data["Max Temp"] > 70) &
                        (city_data["Wind Speed"] < 10) &
                        (city_data["Cloudiness"] == 0)]

#Reset index and drop null values
vacation = vacation.reset_index(drop=True)
vacation = vacation.dropna()
vacation

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,hambantota,0,LK,1585015854,100,6.12,81.12,73.40,3.36
1,morondava,0,MG,1585015862,93,-20.28,44.28,72.68,1.99
2,pauini,0,BR,1585015627,92,-7.71,-66.98,70.70,2.13
3,arraial do cabo,0,BR,1585015870,65,-22.97,-42.03,75.92,9.86
4,quang ngai,0,VN,1585015866,72,15.12,108.80,79.52,6.04
5,villarrica,0,PY,1585015916,52,-25.75,-56.43,73.94,4.36
6,mana,0,GF,1585015916,86,5.66,-53.78,74.66,3.53
7,ndele,0,CF,1585015933,26,8.41,20.65,73.94,2.10
8,san lazaro,0,BZ,1585015936,91,18.04,-88.66,70.88,3.67


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
#Store new dataframe as hotel_df
hotel_df = vacation
hotel_df['Hotel Name'] = ""
hotel_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,hambantota,0,LK,1585015854,100,6.12,81.12,73.40,3.36,
1,morondava,0,MG,1585015862,93,-20.28,44.28,72.68,1.99,
2,pauini,0,BR,1585015627,92,-7.71,-66.98,70.70,2.13,
3,arraial do cabo,0,BR,1585015870,65,-22.97,-42.03,75.92,9.86,
4,quang ngai,0,VN,1585015866,72,15.12,108.80,79.52,6.04,
5,villarrica,0,PY,1585015916,52,-25.75,-56.43,73.94,4.36,
6,mana,0,GF,1585015916,86,5.66,-53.78,74.66,3.53,
7,ndele,0,CF,1585015933,26,8.41,20.65,73.94,2.10,
8,san lazaro,0,BZ,1585015936,91,18.04,-88.66,70.88,3.67,


In [7]:
#Set base URL
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

#Establish what we're searching for
#Check out https://developers.google.com/places/web-service/supported_types to see what I can search for
target_type = "lodging"
target_radius = 5000

#Set Parameters
params = {"radius": target_radius,
          "type": target_type,
          "key": gkey}

#Iterate through dataframe
for index, row in hotel_df.iterrows():
    city = row["City"]
    
    #Get lat and lng and create new parameter so it changes with each city
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    #Assemble URL and make API request
    print(f"Retrieving results for Index {index}: {city}.")
    response = requests.get(base_url, params=params).json()
    
    #Get results
    results = response['results']
    try:
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']
        print(f"The first hotel for {city} is: {results[0]['name']}.")
    except Exception as e:
        print("Missing field/result... skipping")
        pass
    print("---------------------")

Retrieving results for Index 0: hambantota.
The first hotel for hambantota is: Bungalow 63.
---------------------
Retrieving results for Index 1: morondava.
The first hotel for morondava is: La Case Bambou.
---------------------
Retrieving results for Index 2: pauini.
The first hotel for pauini is: Casa do Kiwliks.
---------------------
Retrieving results for Index 3: arraial do cabo.
The first hotel for arraial do cabo is: Pousada Porto Praia.
---------------------
Retrieving results for Index 4: quang ngai.
The first hotel for quang ngai is: Central Hotel.
---------------------
Retrieving results for Index 5: villarrica.
The first hotel for villarrica is: Mansión Reljuv Tours SA.
---------------------
Retrieving results for Index 6: mana.
The first hotel for mana is: LE SAMANA.
---------------------
Retrieving results for Index 7: ndele.
Missing field/result... skipping
---------------------
Retrieving results for Index 8: san lazaro.
Missing field/result... skipping
----------------

In [8]:
hotel_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,hambantota,0,LK,1585015854,100,6.12,81.12,73.40,3.36,Bungalow 63
1,morondava,0,MG,1585015862,93,-20.28,44.28,72.68,1.99,La Case Bambou
2,pauini,0,BR,1585015627,92,-7.71,-66.98,70.70,2.13,Casa do Kiwliks
3,arraial do cabo,0,BR,1585015870,65,-22.97,-42.03,75.92,9.86,Pousada Porto Praia
4,quang ngai,0,VN,1585015866,72,15.12,108.80,79.52,6.04,Central Hotel
5,villarrica,0,PY,1585015916,52,-25.75,-56.43,73.94,4.36,Mansión Reljuv Tours SA
6,mana,0,GF,1585015916,86,5.66,-53.78,74.66,3.53,LE SAMANA
7,ndele,0,CF,1585015933,26,8.41,20.65,73.94,2.10,
8,san lazaro,0,BZ,1585015936,91,18.04,-88.66,70.88,3.67,


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [10]:
# Add marker layer ontop of heat map
hotel_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(hotel_layer)

# Display Map
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…